## Практическая работа

В этой практической работе пять обязательных задач.

*Обязательные задачи* помогут проверить, что вы действительно усвоили материал модуля. Сдайте их на проверку.

Удачи!

Цели практической работы: 


1.   Потренироваться в обучении модели линейной регрессии.
2.   Потренироваться в обучении модели логистической регрессии. 
3.   Потренироваться в обучении модели многослойного персептрона.





Что оценивается:

*   Все пункты и критерии приёмки по каждому заданию выполнены.
*   Код оформлен и работает корректно.





Как отправить работу на проверку:

Сдайте практическую работу этого модуля через систему контроля версий Git сервиса Skillbox GitLab. После загрузки работы на проверку напишите об этом в личном кабинете своему куратору.

В этом задании вам нужно построить модель классификации, определяющую категорию цены/цену подержанного автомобиля в зависимости от характеристик транспортного средства. 

Мы будем использовать датасет из коллекции подержанных автомобилей, выставленных на продажу в Соединенных Штатах. Он уже подготовлен, без выбросов и с категориальными фичами, преобразованными с помощью one hot encoding, и количественными фичами, стандартизированными с помощью скейлеров.


### Описание датасета:
- `id`: идентификатор записи;
- `is_manufacturer_name`: признак производителя автомобиля;

- `region_*`: регион;
- `x0_*`: тип топлива;
- `manufacturer_*`: производитель;
- `short_model_*`: сокращённая модель автомобиля;
- `title_status_*`: статус;
- `transmission_*`: коробка передач;
- `state_*`: штат;
- `age_category_*`: возрастная категория автомобиля;

- `std_scaled_odometer`: количество пройденных миль (после стандартизации);
- `year_std`: год выпуска (после стандартизации);
- `lat_std`: широта (после стандартизации);
- `long_std`: долгота (после стандартизации);
- `odometer/price_std`: отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`: количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`: количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`: длина наименования автомобиля (после стандартизации);
- `model_word_count_std`: количество слов в наименовании автомобиля (после стандартизации);
- `month_std`: номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`: день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`: количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`: стоимость;
- `price_category`: категория цены.

## Обязательные задачи

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('vehicles_dataset_prepared.csv')
df.head()

,id,price,price_category,is_audi,is_ford,is_chevrolet,is_toyota,x0_diesel,x0_electric,x0_gas,...,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std,model_word_count_std,month_std,dayofweek_std,diff_years_std
0,7308295377,54990,high,0,0,0,0,1.0,0.0,0.0,...,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032,1.910669,-0.615846,1.120284,-1.322394
1,7316380095,16942,medium,0,1,0,0,0.0,0.0,0.0,...,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087,1.235799,1.623784,-1.374972,-0.695973
2,7313733749,35590,high,0,0,0,0,0.0,0.0,1.0,...,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197,0.560930,-0.615846,-0.376870,-0.852578
3,7308210929,14500,medium,0,0,0,1,0.0,0.0,1.0,...,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473,-0.788810,-0.615846,1.120284,-0.226157
4,7303797340,14590,medium,0,0,0,0,0.0,0.0,0.0,...,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032,1.910669,-0.615846,0.122182,-0.069552


**Задача 0. Линейная регрессия**

Вспомните задачу по предсказанию стоимости поддержанного автомобиля. Попробуем обучить модель линейной регрессии для предсказания цены автомобиля (колонка `price`). Для этого проделайте следующие шаги:

- Подготовьте данные: удалите колонки, которые косвенно содержат информацию о целевой переменной (`odometer/price_std`, `price_category`).
- Разделите выборку на треин и тест в отношении 70/30.
- Обучите модель линейной регрессии с дефолтными параметрами.
- Посчитайте значение метрики mae на тестовой выборке для линейной регрессии.
- Выведите получившиеся коэффициенты линейной регрессии при каждом параметре обучающей выборки с помощью метода `coef_`. Есть ли коэффициенты, которые равны нулю? Если есть, выведите названия фичей с нулевым коэффициентом
- Удалите фичи, коэффициенты которых равны нулю. И переобучите модель. Убедитесь, что значение метрики не изменилось.



In [3]:
# Ваш код здесь
df_prepared = df.copy()
df_prepared = df_prepared.drop(['price_category', 'odometer/price_std'], axis=1)
train,test = train_test_split(df_prepared,test_size = 0.3, random_state=42)
lr = LinearRegression()
lr.fit(train.drop(['price'], axis=1), train.price)
pred = lr.predict(test.drop(['price'], axis=1))
mean_absolute_error(test.price, pred)

4600.341115781948

In [4]:
features_list = list(train.drop(['price'], axis=1))
features_list


['id',
 'is_audi',
 'is_ford',
 'is_chevrolet',
 'is_toyota',
 'x0_diesel',
 'x0_electric',
 'x0_gas',
 'x0_hybrid',
 'x0_other',
 'std_scaled_odometer',
 'short_model_-benz',
 'short_model_1',
 'short_model_124',
 'short_model_128i',
 'short_model_135i',
 'short_model_1500',
 'short_model_163869',
 'short_model_1928',
 'short_model_1951',
 'short_model_1954',
 'short_model_1960s',
 'short_model_1966',
 'short_model_1968',
 'short_model_1972',
 'short_model_1977',
 'short_model_1978',
 'short_model_198.5',
 'short_model_1988',
 'short_model_1994',
 'short_model_1999',
 'short_model_2',
 'short_model_200',
 'short_model_2000',
 'short_model_2002',
 'short_model_2006',
 'short_model_2007',
 'short_model_2008',
 'short_model_2009',
 'short_model_2012',
 'short_model_2015',
 'short_model_2016',
 'short_model_2017',
 'short_model_2018',
 'short_model_210',
 'short_model_240',
 'short_model_2500',
 'short_model_2500hd',
 'short_model_280zx',
 'short_model_3',
 'short_model_3-series',
 'short

In [5]:
for col, i in zip(features_list, lr.coef_):
    if i == 0.0:
        print(f'Нулевой коэф {col}={i}')

Нулевой коэф manufacturer_ferrari=0.0


In [6]:
df_prepared = df.drop(columns = ['odometer/price_std','price_category','manufacturer_ferrari'])
train,test = train_test_split(df_prepared,test_size = 0.3, random_state=42)
lr = LinearRegression()
lr.fit(train.drop(['price'], axis=1), train.price)
pred = lr.predict(test.drop(['price'], axis=1))
mean_absolute_error(test.price, pred)

4600.341115918074

**Задача 1. Логистическая регрессия**

Теперь в рамках тех же данных попробуйте предсказать `price_category` с помощью алгоритма логистической регрессии. Предварительно из датафрейма удалите переменные, в которых косвенно содержится информация о целевой переменной (`odometer/price_std`, `price`). 

Для обученной модели нужно:

- рассчитать и вывести метрику качества (accuracy) на тренировочной выборке;
- сделать предикт на тестовых данных и положить его в переменную `logreg_pred`;
- рассчитать и вывести accuracy и confusion_matrix на тестовой выборке.


Задание засчитывается, если: 
- accuracy на тренировочной выборке > 87%;
- accuracy на тестовой выборке > 75.5%.

Для улучшения метрики можно преобразовывать входные данные, тюнить параметры логистической регрессии.

In [7]:
df_logr = df.drop(['price', 'odometer/price_std','id'], axis=1)

In [8]:
# Ваш код подготовки данных здесь. Допишите инициализацию входных фичей и целевую переменную
x = df_logr.drop(['price_category'], axis=1)
y = df_logr.price_category

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [9]:
# Ваш код обучения моделей здесь
logreg = LogisticRegression(max_iter=1000)
logreg.fit(x_train,y_train)


LogisticRegression(max_iter=1000)

In [10]:
preg = logreg.predict(x_train)
accuracy_score(y_train,preg)

0.8550423288281598

In [11]:
log_preg = logreg.predict(x_test)
accuracy_score(y_test,log_preg)

0.7571032571032571

In [12]:
logreg_tune = LogisticRegression(max_iter=1000,C=10)
logreg_tune.fit(x_train,y_train)


LogisticRegression(C=10, max_iter=1000)

In [13]:
preg_tune = logreg_tune.predict(x_train)
accuracy_score(y_train,preg_tune)

0.893361057478093

In [14]:
log_preg_tune = logreg_tune.predict(x_test)
accuracy_score(y_test,log_preg_tune)

0.7633402633402634

**Задача 2. Многослойный персептрон**

Решите задачу, поставленную в предыдущем задании, применив модель многослойного персептрона. Добейтесь лучших значений точности на тренировочной и тестовой выборках, чем на логистической регрессии, с помощью данного алгоритма. 

Значимым увеличением точности в данном задании будем считать увеличение больше, чем на 0.01 по метрике accuracy.

In [15]:
df_logr = df.drop(['price', 'odometer/price_std','id'], axis=1)
x = df_logr.drop(['price_category'], axis=1)
y = df_logr.price_category

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [89]:
# Ваш код здесь
mlp = MLPClassifier(random_state=30, max_iter=50, solver='adam',alpha=0.0001,hidden_layer_sizes=20)
mlp.fit(x_train,y_train)

D:\Ana\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=20, max_iter=50, random_state=30)

In [42]:
parameters = {'solver': ['lbfgs'], 'max_iter': [1000,1900,2000 ], 'hidden_layer_sizes':np.arange(10, 15)}
mlp_clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
mlp_clf.fit(x_train, y_train)


GridSearchCV(estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'hidden_layer_sizes': array([10, 11, 12, 13, 14]),
                         'max_iter': [1000, 1900, 2000], 'solver': ['lbfgs']})

In [43]:
best_params_mlp = mlp_clf.best_params_
best_params_mlp

{'hidden_layer_sizes': 13, 'max_iter': 2000, 'solver': 'lbfgs'}

In [90]:
mlp_pred= mlp.predict(x_test)
accuracy_score(y_test, mlp_pred)

0.7706167706167706

In [91]:
mlp_pred2 = mlp.predict(x_train)
accuracy_score(y_train,mlp_pred2)

0.9322738749443041

**Задача 3. Сравнение с древовидными моделями**

Обучите модель случайного леса на тех же данных для предсказания `price_category`. Сравните качество с моделью логистической регрессии и многослойного персептрона. Словами опишите, какая из моделей в каких случаях работает лучше по результатам на тестовой выборке, и обоснуйте свой выбор.

In [19]:
# Ваш код здесь
from sklearn.ensemble import RandomForestClassifier

In [20]:
df_logr = df.drop(['price', 'odometer/price_std','id'], axis=1)
x = df_logr.drop(['price_category'], axis=1)
y = df_logr.price_category

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

In [21]:
rf_clf = RandomForestClassifier(max_features='log2', min_samples_leaf=1, n_estimators=300)
rf_clf.fit(x_train,y_train)

RandomForestClassifier(max_features='log2', n_estimators=300)

In [165]:
param_grid = {
    'n_estimators': list(range(100,501,100)),
    'max_features': ['sqrt', 'log2'],
    'min_samples_leaf': list(range(1,10))
}    

In [166]:
grid_search_rf_old= GridSearchCV(
    estimator = rf_clf,
    param_grid=param_grid,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

In [167]:
grid_search_rf_old.fit(x_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='accuracy', verbose=1)

In [168]:
best_params = grid_search_rf_old.best_params_
best_params

{'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 300}

In [22]:
predicted_test_rf = rf_clf.predict(x_test)
predicted_train_rf = rf_clf.predict(x_train)
print(accuracy_score(y_test,predicted_test_rf))
print(accuracy_score(y_train,predicted_train_rf))

0.7598752598752598
1.0


**Задача 4. Стратегия голосования**

Реализуйте стратегию голосования для предсказания целевой переменной.
Голосование в задаче классификации — это когда несколько моделей выдают свои предикты, и финальным выбирается тот предикт, который предсказали большинство моделей.

Для реализации этой стратегии проделайте следующее:

- сохраните предсказания каждой из моделей (случайный лес, многослойный персептрон, логистическая регрессия) для тестовой выборки в датафрейм `pred_df`;
- в четвертую колонку `target` положите тот класс, который предсказало большинство классификаторов. Например, если в строке были значения `high, medium, medium`, в `target` нужно положить `medium`.

     Если в строке три разных класса — `high, medium, low`, — придумайте свою стратегию по выбору значения. Самая простая стратегия: выбрать рандомно одно значение из трёх.

- посчитайте точность предсказания с помощью голосования. Выведите значения метрик accuracy и confusion_matrix.


Добейтесь значения точности > 78% 

In [93]:
# Ваш код здесь
pred_df = pd.DataFrame({'RandomForest':predicted_test_rf,'MultilayerPerceptron':mlp_pred,'LogisticPerpession':log_preg_tune})
pred_df

,RandomForest,MultilayerPerceptron,LogisticPerpession
0,high,high,high
1,medium,medium,medium
2,low,low,low
3,low,low,low
4,high,high,high
...,...,...,...
2881,medium,medium,medium
2882,low,low,low
2883,low,low,low
2884,medium,medium,medium


In [94]:
pred_df['target'] = pred_df.apply(lambda x: x.mode()[0],axis=1)
pred_df

,RandomForest,MultilayerPerceptron,LogisticPerpession,target
0,high,high,high,high
1,medium,medium,medium,medium
2,low,low,low,low
3,low,low,low,low
4,high,high,high,high
...,...,...,...,...
2881,medium,medium,medium,medium
2882,low,low,low,low
2883,low,low,low,low
2884,medium,medium,medium,medium


In [96]:
df_logr = df.drop(['price', 'odometer/price_std','id'], axis=1)
x = df_logr.drop(['price_category'], axis=1)
y = df_logr.price_category

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=42)

In [95]:
accuracy_score(y_test,pred_df['target'])

0.7737352737352737

In [27]:
confusion_matrix(y_test,pred_df.target)

array([[849,  30, 118],
       [ 70, 770, 113],
       [221, 163, 552]], dtype=int64)

**Примечание:**

В этом домашнем задании в задаче номер 1 и далее перед вами встал выбор: включать ли в обучение колонку `id`. При удалении данной колонки во время обучении логистической регрессии качество заметно улучшается.

Вообще обучать любую модель (будь то логистическая или линейная регрессия, или древовидный алгоритм) на данных айдишников не считается хорошей практикой. Как правило, между `id` и целевой переменной не должно быть никаких взаимосвязей. Включая колонку `id` в качества атрибута в обучение, вы стараетесь "заточить" результаты своей модели под айдишники записей. Таким образом модель обучится на некотором наборе "частных случаев", и, возможно, не обратит внимание на общие зависимости.   